# Load Cases

## 1. Like working with different currencies

Totals (or sub-totals) need to be collected for each load category separately.

In [92]:
DL_2 = 25
LL_2 = 30
SL_2 = 15
WL_2 = 10

DL_1 = 25
LL_1 = 60

DL = DL_1 + DL_2
LL = LL_1 + LL_2
SL = SL_2
WL = WL_2
print(f"{DL=}, {LL=}, {SL=}, {WL=}")

DL=50, LL=90, SL=15, WL=10


## 2. Factoring allows for conversion to "a common currency"

In order to see if there is enough money in the account (member capacity) to pay for the expenses (load demand), we need to convert to a "common currency" by factoring.

In [93]:
FL = 1.25*DL + 1.5*LL + 1.0*SL
print(f"{FL=}")

FL=212.5


## 3. Managing the many possible combinations of factored loads

### A. Separate functions

Pros: 
* Once written, each function has its own unique name can be called easily with basic inputs (i.e. just the loads)
* Easy to write

Cons:
* Lots of repetitive code to write
* Might be difficult to maintain, easy to introduce bugs through over repetition
* Function signatures are highly variable (e.g. some funcs only take a `DL` argument, some take `DL`, `LL`, and `SL`. How to organize that?

In [94]:
def LC1(DL: float) -> float:
    return 1.4 * DL

def LC2(DL: float, LL: float) -> float:
    return 1.25 * DL + 1.5 * LL

def LC2a(DL: float, LL: float, SL: float) -> float:
    return 1.25 * DL + 1.5 * LL + 1.0 * SL

def LC2b(DL: float, LL: float, WL: float) -> float:
    return 0.9 * DL + 1.5 * LL + 0.4 * WL

def LC2c(DL: float, LL: float, WL: float) -> float:
    return 1.25 * DL + 1.5 * LL + 0.4 * WL
    

In [95]:
LC2b(DL=DL, LL=LL, WL=WL)

184.0

## B. A generic function

Pros: 
* Write just one function to handle all cases
* Flexible to handle any inputs
* Load combination factors can be stored in an external file: easy to maintain
* Extensible

Cons: 
* Calling the function could get repetitive and tedious, especially if being used in an interactive setting.
* Bad inputs and errors can occur in calling the function.

In [96]:
def factor_loads(
    DL_factor: float = 0,
    DL: float = 0,
    LL_factor: float = 0,
    LL: float = 0,
    SL_factor: float = 0,
    SL: float = 0,
    WL_factor: float = 0,
    WL: float = 0,
    EL_factor: float = 0,
    EL: float = 0,
) -> float:
    """
    Returns the factored load for the given load factors
    and loads.
    """
    factored_load = (
        DL_factor * DL
        + LL_factor * LL
        + SL_factor * SL
        + WL_factor * WL
        + EL_factor * EL
    )
    return factored_load

In [97]:
lc2 = factor_loads(DL_factor=1.25, DL=DL, LL_factor=1.5, LL=LL)
lc2a = factor_loads(
    DL_factor = 1.25, 
    DL=DL, 
    LL_factor = 1.5, 
    LL=LL, 
    SL_factor=1.0, 
    SL=SL
)
print(f"{lc2=}, {lc2a=}")

lc2=197.5, lc2a=212.5


## C. Vector math

Pros: 
* Fast
* Flexible for different inputs
* Load combination vectors can be stored in an external file: easy to maintain.

Cons: 
* Arrays are positional but do not have "labelled positions"
* If you need to add an array position, the corresponding arrays need to be updated across all functions (less extensible)

In [98]:
import numpy as np

LC1 = [1.4, 0, 0, 0, 0]
LC2 = [1.25, 1.5, 0, 0, 0]
LC2a = [1.25, 1.5, 1.0, 0, 0]
LC2b = [0.9, 1.5, 0, 0.4, 0]

load = [DL, LL, SL, WL, 0]

def factor_vec_load(load_vector: np.ndarray, factor_vector: np.ndarray) -> float:
    """
    Returns the loads in 'load_vector' factored by 'factor_vector'.
    Both arrays must be the same length.
    """
    return sum(load_vector * factor_vector)
    
fl = factor_vec_load(np.array(load), np.array(LC2a))

print(f"{fl=}")


fl=212.5


## 4. Storing Load Combinations

In [99]:
import json

### Factor Dictionaries

```json
{
    "LC1": {
        "DL_factor": 1.4
    },
    "LC2": {
        "DL_factor": 1.25,
        "LL_factor": 1.5
    },
    "LC2a": {
        "DL_factor": 0.9,
        "LL_factor": 1.5
    },
    "LC2b": {
        "DL_factor": 1.25,
        "LL_factor": 1.5,
        "SL_factor": 1.0
    },
    "LC2c": {
        "DL_factor": 1.25,
        "LL_factor": 1.5,
        "WL_factor": 1.0
    },
    "LC2d": {
        "DL_factor": 0.9,
        "LL_factor": 1.5,
        "WL_factor": 0.4
    }
}
```

In [100]:
with open("NBCC_dict.json", 'r') as json_file:
    nbcc_dict = json.load(json_file)
nbcc_dict

{'LC1': {'DL_factor': 1.4},
 'LC2': {'DL_factor': 1.25, 'LL_factor': 1.5},
 'LC2a': {'DL_factor': 0.9, 'LL_factor': 1.5},
 'LC2b': {'DL_factor': 1.25, 'LL_factor': 1.5, 'SL_factor': 1.0},
 'LC2c': {'DL_factor': 1.25, 'LL_factor': 1.5, 'WL_factor': 1.0},
 'LC2d': {'DL_factor': 0.9, 'LL_factor': 1.5, 'WL_factor': 0.4}}

### Vector dictionaries

```json
{
    "LC1": [1.4, 0, 0, 0, 0],
    "LC2": [1.25, 1.5, 0, 0, 0],
    "LC2a": [0.9, 1.5, 0, 0, 0],
    "LC2b": [1.25, 1.5, 1.0, 0, 0],
    "LC2c": [1.25, 1.5, 0, 0.4, 0],
    "LC2d": [0.9, 1.5, 0.4, 0, 0],
}
```

In [101]:
with open('NBCC_vec.json', 'r') as json_file:
    nbcc_vec = json.load(json_file)
nbcc_vec

{'LC1': [1.4, 0, 0, 0, 0],
 'LC2': [1.25, 1.5, 0, 0, 0],
 'LC2a': [0.9, 1.5, 0, 0, 0],
 'LC2b': [1.25, 1.5, 1.0, 0, 0],
 'LC2c': [1.25, 1.5, 0, 0.4, 0],
 'LC2d': [0.9, 1.5, 0.4, 0, 0]}

In [102]:
print(f'{factor_loads(DL=2.4, LL=0.9, SL=3.6, **factor_dict["LC2a"])=}')
print(f'{factor_vec_load(np.array([2.4, 0.9, 3.6, 0, 0]), np.array(vector_dict["LC2a"]))=}')

factor_loads(DL=2.4, LL=0.9, SL=3.6, **factor_dict["LC2a"])=3.5100000000000002
factor_vec_load(np.array([2.4, 0.9, 3.6, 0, 0]), np.array(vector_dict["LC2a"]))=3.5100000000000002


## 4. Load Categories

> Sometimes there are two load "categories" that we may want to treat as being a part of the same load "case".
>
> e.g. Both self-weight and super-imposed dead load can be considered "dead load" or `DL`

In [103]:
def transform_to_service_loads(loads: dict, load_categories: dict) -> dict:
    """
    Returns a dictionary of service loads (i.e. loads conforming to the
    NBCC categories of DL, LL, SL, etc.) obtained from the given loads
    in 'loads' correlated against the load types in 'load_categories'.
    
    If the load type in 'loads' is not listed in 'load_categories', no 
    transformation takes place.
    """
    service_loads = {} # acc
    for load_name, load in loads.items():
        load_type = load_categories.get(load_name, load_name)
        if load_type in service_loads:
            service_loads[load_type] = service_loads[load_type] + load
        else:
            service_loads.update({load_type: load})
    return service_loads

### Example

load_categories_lookup = {
    "SW": "DL",
    "SDL": "DL",
    "Planter Soil": "LL",
    "Snow": "SL", 
    "Rain": "SL"
}
loads = {"SW": 14, "SDL": 30, "Planter Soil": 180, "Snow": 30, "Rain": 2}

service_loads = transform_to_service_loads(loads, load_categories_lookup)
print(f"{loads=}")
print(f"{service_loads=}")

loads={'SW': 14, 'SDL': 30, 'Planter Soil': 180, 'Snow': 30, 'Rain': 2}
service_loads={'DL': 44, 'LL': 180, 'SL': 32}


## 4. Find maximum factored load

### A. Maximum value only (both dict and vector examples)

In [104]:
def max_factored_dict(load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**load, **load_factors)
        fl = max(current_fl, fl)
    return fl

### Example
service_loads = {'DL': 44, 'LL': 180, 'SL': 32}
print("Dictionary example: ", f"{service_loads=}")

max_factored = max_factored_dict(service_loads, nbcc_dict)
print(f"{max_factored=}", "\n")


def max_factored_vec(load: list, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    for load_combination, load_factors in load_combinations.items():
        load_vector = np.array(load)
        factor_vector = np.array(load_factors)
        current_fl = factor_vec_load(load_vector, factor_vector)
        fl = max(current_fl, fl)
    return fl

### Example
service_loads = [44, 180, 32, 0, 0]
print("Vector example: ", f"{service_loads=}")
max_factored = max_factored_vec(service_loads, nbcc_vec)
print(f"{max_factored=}")

Dictionary example:  service_loads={'DL': 44, 'LL': 180, 'SL': 32}
max_factored=357.0 

Vector example:  service_loads=[44, 180, 32, 0, 0]
max_factored=357.0


### B. Maximum value and governing combination (both dict and vector examples)

In [105]:
def max_factored_dict_govern(load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**load, **load_factors)
        if current_fl > fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc

### Example

service_loads = {'DL': 44, 'LL': 180, 'SL': 32}
print("Dictionary example: ", f"{service_loads=}")

max_factored = max_factored_dict_govern(service_loads, nbcc_dict)
print(f"{max_factored=}", "\n")
fl, lc = max_factored
print(f"{service_loads=}\n{fl=}\n{nbcc_dict[lc]=}\n\n")


def max_factored_vec_govern(load: list, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        load_vector = np.array(load)
        factor_vector = np.array(load_factors)
        current_fl = factor_vec_load(load_vector, factor_vector)
        if current_fl > fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc

### Example

service_loads = [44, 180, 32, 0, 0]
print("Vector example: ", f"{service_loads=}")
max_factored = max_factored_vec_govern(service_loads, nbcc_vec)
print(f"{max_factored=}\n")
fl, lc = max_factored
print(f"{service_loads=}\n{fl=}\n{nbcc_vec[lc]=}")

Dictionary example:  service_loads={'DL': 44, 'LL': 180, 'SL': 32}
max_factored=(357.0, 'LC2b') 

service_loads={'DL': 44, 'LL': 180, 'SL': 32}
fl=357.0
nbcc_dict[lc]={'DL_factor': 1.25, 'LL_factor': 1.5, 'SL_factor': 1.0}


Vector example:  service_loads=[44, 180, 32, 0, 0]
max_factored=(357.0, 'LC2b')

service_loads=[44, 180, 32, 0, 0]
fl=357.0
nbcc_vec[lc]=[1.25, 1.5, 1.0, 0, 0]


## 5. Occupancies

> In buildings, loads can often be grouped together in different occupancies.
>
> `dict`s can be useful for organizing these.

In [106]:
import forallpeople as si
si.environment('structural')

### A. Occupancy dictionaries

In [107]:
occupancies_dict = {
    "Residential": {
        "DL": 2.4 * si.kPa,
        "LL": 2.4 * si.kPa,
    },
    "Residential Roof": {
        "DL": 0.9 * si.kPa,
        "SL": 1.3 * si.kPa,
    },
    "Residential Amenity Roof": {
        "DL": 2.6 * si.kPa,
        "LL": 4.8 * si.kPa,
    },
    "Rooftop Mech": {
         "DL": 1.83 * si.kPa,
     },
}

def multiply_value(occupancy: dict, value: float) -> dict:
    """
    Multiplies 'value' by each of the loads in 'occupancy'.
    """
    acc = {}
    for load_type, load in occupancy.items():
        acc.update({load_type: load * value})
    return acc

In [108]:
forces = multiply_value(occupancies_dict["Residential Amenity Roof"], 10*si.m**2)
print(f"{forces=}")

forces={'DL': 26.000 kN, 'LL': 48.000 kN}


### B. Occupancy vectors

In [109]:
occupancies_vec = {
    "Residential": np.array([2.4, 2.4, 0, 0, 0]) * si.kPa,
    "Residential Roof": np.array([0.9, 0, 1.3, 0, 0]) * si.kPa,
    "Residential Amenity Roof": np.array([2.6, 4.8, 0, 0, 0]) * si.kPa,
    "Rooftop Mech": np.array([1.83, 0, 0, 0, 0]) * si.kPa,
}

In [110]:
forces = occupancies_vec["Residential Amenity Roof"] * 10*si.m**2
print(f"{forces=}")

forces=array([26.000 kN, 48.000 kN, 0.000 N, 0.000 N, 0.000 N], dtype=object)
